# 02｜交易成本与偏差：前视偏差、成本与滑点的影响

目的：本笔记本用 AAPL 日线数据演示前视偏差与交易成本对回测结果的影响，给出极简、可复现实验。

金融含义：真实交易受执行时点与成本影响，naive 回测往往高估绩效；正确对齐与成本建模能更接近实盘，认识换手与成本的关系是量化基础。


## 导入与数据下载

- 目的：导入必要库，下载 AAPL（2015-01-01 至今）并取收盘价 `px`。
- 金融含义：保证数据来源一致与可重复，使用复权价格更贴近可交易价格序列。


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime

pd.options.display.float_format = '{:.4f}'.format
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

ok = True
try:
    data = yf.download('AAPL', start='2015-01-01', end=datetime.now().strftime('%Y-%m-%d'), auto_adjust=True)
    if data is None or len(data) == 0:
        raise ValueError('empty data')
    px = data['Close'].dropna().rename('px')
except Exception as e:
    ok = False
    print(f'数据下载失败：{e}')


## 信号与 naive vs. correct 回测

- 目的：构建 10/50 均线金叉信号，并对比 naive 前视与正确对齐两种回测。
- 金融含义：信号必须用于下一期执行（`shift(1)`），否则会引入前视偏差而高估业绩。


In [ ]:
if not ok:
    raise SystemExit

ret = px.pct_change().fillna(0)
sma10 = px.rolling(10).mean()
sma50 = px.rolling(50).mean()
signal = (sma10 > sma50).astype(int)
pos = signal.shift(1).fillna(0).astype(int)

# naive（前视偏差）
ret_naive = signal * ret
# correct（无前视）
ret_correct = pos * ret

# 净值
equity_naive = (1 + ret_naive).cumprod()
equity_correct = (1 + ret_correct).cumprod()


## 加入交易成本与滑点

- 目的：在正确回测基础上叠加佣金、滑点与半价差，构造净收益（含成本）。
- 金融含义：成本按换手率计提，换手越高成本越高；bps 需换算为比例 `bps/1e4`。


In [ ]:
# 成本超参数（单边，单位：bps）
commission_bps = 5
slippage_bps = 2
half_spread_bps = 5

turnover = pos.diff().abs().fillna(0)
cost_bps = commission_bps + slippage_bps + half_spread_bps
cost = turnover * (cost_bps / 1e4)

ret_correct_cost = pos * ret - cost

equity_correct_cost = (1 + ret_correct_cost).cumprod()


## 净值曲线与指标表

- 目的：同图对比三条净值曲线，计算并展示年化、Sharpe、最大回撤。
- 金融含义：naive 高估；正确对齐后回落；加入成本进一步下降，幅度与换手相关。


In [ ]:
def stats_from_ret(r):
    eq = (1 + r).cumprod()
    n = len(eq)
    if n == 0:
        return {'CAGR': np.nan, 'Vol': np.nan, 'Sharpe': np.nan, 'MaxDD': np.nan}
    years = n / 252
    cagr = eq.iloc[-1] ** (1/years) - 1 if years > 0 else np.nan
    vol = r.std(ddof=0) * np.sqrt(252)
    sharpe = (r.mean() / (r.std(ddof=0) + 1e-12)) * np.sqrt(252)
    dd = eq / eq.cummax() - 1
    maxdd = dd.min()
    return {'CAGR': cagr, 'Vol': vol, 'Sharpe': sharpe, 'MaxDD': maxdd}

m_naive = stats_from_ret(ret_naive)
m_corr = stats_from_ret(ret_correct)
m_cost = stats_from_ret(ret_correct_cost)

# 紧凑表格
res = pd.DataFrame([m_naive, m_corr, m_cost], index=['naive', 'correct', 'correct_cost'])
print(res.applymap(lambda x: np.nan if pd.isna(x) else round(float(x), 4)))

# 净值曲线对比
plt.figure(figsize=(12,6))
plt.plot(equity_naive.index, equity_naive.values, label='equity_naive', linewidth=1.6)
plt.plot(equity_correct.index, equity_correct.values, label='equity_correct', linewidth=1.6)
plt.plot(equity_correct_cost.index, equity_correct_cost.values, label='equity_correct_cost', linewidth=1.8)
plt.title('AAPL 动量策略：naive vs. correct vs. correct_cost')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 成本影响（条形图）
vals = res.loc[['correct','correct_cost'], ['CAGR','Sharpe']]
delta = (vals.loc['correct'] - vals.loc['correct_cost']).rename('drop')
plt.figure(figsize=(8,4))
plt.bar(delta.index, delta.values, color=['tab:orange','tab:purple'])
plt.title('加入成本后的指标下降幅度')
for i, v in enumerate(delta.values):
    plt.text(i, v, f'{v:.3f}', ha='center', va='bottom' if v>=0 else 'top')
plt.grid(True, axis='y', alpha=0.3)
plt.tight_layout()
plt.show()


## （可选）极简限价单填单逻辑

- 目的：示例“逢低买入”限价下单与次日撮合的最小实现，统计填单率与收益对比。
- 金融含义：限价单能改善成交价格但存在不成交风险；成交概率与价差设置、波动有关。


In [ ]:
# 逢低买入：t日 0->1 时，在 t 收盘生成限价，t+1 看区间是否成交
low = data['Low']
high = data['High']
prev_close = px.shift(1)

sig_up = (signal.diff() == 1).astype(int)  # 0->1 切换点
limit_gap = 0.002  # 0.2%
limit_price = (prev_close * (1 - limit_gap)).where(sig_up==1)

# 次日撮合
hit = ((low <= limit_price) & (high >= limit_price)).fillna(False)
# 限价成交的持仓路径（只示例进场，出场仍按信号转回0）
pos_lmt = pos.copy()
pos_lmt[(sig_up==1) & (~hit)] = 0  # 未成交则不持仓

ret_lmt = pos_lmt.shift(1).fillna(0) * ret  # 下一期执行
turnover_lmt = pos_lmt.diff().abs().fillna(0)
ret_lmt_cost = ret_lmt - turnover_lmt * (cost_bps/1e4)

m_lmt = stats_from_ret(ret_lmt_cost)
print({k: (np.nan if pd.isna(v) else round(float(v),4)) for k,v in m_lmt.items()})
print('fill_rate', round(hit.mean(),4), 'avg_turnover', round(turnover_lmt.mean(),4))


## （可选）幸存者偏差最小实验（合成数据）

- 目的：展示“剔除历史最差20%资产”会虚增回测表现。
- 金融含义：真实研究需用当期成分快照，避免只看幸存者导致高估绩效。


In [ ]:
np.random.seed(0)
N, T = 100, 252*3
rets = np.random.normal(0.0004, 0.015, size=(T, N))  # 合成日收益
cum = (1 + pd.DataFrame(rets)).cumprod()
final = cum.iloc[-1]
keep = final.sort_values().iloc[int(N*0.2):].index  # 剔除最差20%

avg_all = rets.mean(axis=1).mean()
avg_keep = pd.DataFrame(rets).iloc[:, keep].mean(axis=1).mean()
print({'avg_all': round(float(avg_all),4), 'avg_keep': round(float(avg_keep),4)})

plt.figure(figsize=(8,4))
plt.bar(['all','drop_worst20%'], [avg_all, avg_keep], color=['gray','tab:green'])
plt.title('幸存者偏差示意：剔除差资产后的平均收益上升')
for i, v in enumerate([avg_all, avg_keep]):
    plt.text(i, v, f'{v:.4f}', ha='center', va='bottom' if v>=0 else 'top')
plt.grid(True, axis='y', alpha=0.3)
plt.tight_layout()
plt.show()


## 要点小结

- 目的：复盘本节关键点，给出实践注意事项。
- 金融含义：正确对齐、成本建模、关注换手，避免前视与幸存者偏差，结果更接近实盘。


- 信号用于下一期执行：`pos = signal.shift(1)` 并处理首日缺失。
- 成本单位统一 bps；换算比例用 `bps/1e4`；成本=换手×单边成本。
- naive 曲线往往偏高；correct 降低；加成本后再降，幅度与换手率相关。
- 限价单可能改善成交价格，但存在不成交风险；参数需慎重回测。
- 幸存者偏差：真实研究需使用当期成分快照，本节实验仅示意原理。


# 第02课：前视偏差与交易成本

本课程将演示量化交易中两个关键问题：前视偏差（Look-ahead Bias）和交易成本的影响。


## 1. 导入库与数据下载

**目的**：下载AAPL日线数据，计算基础收益率序列。

**金融含义**：获取历史价格数据是量化研究的第一步。使用调整后价格可以消除股票分割、分红等事件对价格序列连续性的影响，确保收益率计算的准确性。


In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime

# 设置中文字体和图表样式
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
plt.style.use('default')

print("库导入成功")


库导入成功


In [22]:
# 下载AAPL数据
try:
    data = yf.download('AAPL', start='2015-01-01', end=datetime.now().strftime('%Y-%m-%d'), auto_adjust=True)
    px = data['Close'].dropna()
    ret = px.pct_change().fillna(0)
    print(f"数据下载成功，共{len(px)}个交易日")
    px_series = px['AAPL']
    print(f"价格范围：${px_series.min():.2f} - ${px_series.max():.2f}")
except Exception as e:
    print(f"数据下载失败：{e}")
    print("请检查网络连接后重试")


[*********************100%***********************]  1 of 1 completed

数据下载成功，共2686个交易日
价格范围：$20.62 - $258.10


## 2. 构建动量信号与前视偏差演示

**目的**：构建简单的10/50日均线金叉动量信号，对比有前视偏差和无前视偏差的回测结果。

**金融含义**：前视偏差是量化回测中最常见的错误之一。在实际交易中，我们只能基于已知信息做决策，不能使用"未来"的价格信息。正确的做法是用前一日信号决定今日仓位。


In [23]:
# 构建10/50日均线金叉信号
ma10 = px.rolling(10).mean()
ma50 = px.rolling(50).mean()
signal = (ma10 > ma50).astype(int)  # 1=多仓, 0=空仓

# 前视偏差回测（错误方法）
ret_naive = signal * ret  # 用当日信号交易当日收益
equity_naive = (1 + ret_naive).cumprod()

# 正确回测（无前视偏差）
pos = signal.shift(1).fillna(0)  # 用前一日信号决定今日仓位
ret_correct = pos * ret
equity_correct = (1 + ret_correct).cumprod()

pos_series = pos['AAPL']
print(f"信号总换手次数：{int(signal.diff().abs().sum())}")
print(f"持仓比例：{pos_series.mean():.1%}")


信号总换手次数：69
持仓比例：61.3%


C:\Users\zyf17\AppData\Local\Temp\ipykernel_14052\1760317715.py:16: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  print(f"信号总换手次数：{int(signal.diff().abs().sum())}")


## 3. 加入交易成本分析

**目的**：在正确回测基础上叠加佣金、滑点、买卖价差等交易成本。

**金融含义**：交易成本是实盘交易中不可避免的摩擦成本，包括券商佣金、市场冲击、买卖价差等。高频策略对成本更敏感，忽略成本会严重高估策略收益。换手率越高，成本侵蚀越严重。


In [29]:
# 交易成本参数（单位：基点bps）
commission_bps = 5      # 佣金
slippage_bps = 2       # 滑点
half_spread_bps = 5    # 买卖价差的一半
cost_bps = commission_bps + slippage_bps + half_spread_bps

# 计算换手率和成本
turnover = signal['AAPL'].diff().abs().fillna(0)  # 仓位变化=换手
cost = turnover * (cost_bps / 1e4)     # 换手成本
ret_correct_cost = pos * ret - cost    # 扣除成本后收益
equity_correct_cost = (1 + ret_correct_cost).cumprod()

# 计算年化换手率
annual_turnover = turnover.sum() * 252 / len(turnover)
total_cost_impact = cost.sum()

print(f"单边交易成本：{cost_bps} bps")
print(f"年化换手率：{annual_turnover:.1f}x")
print(f"总成本影响：{total_cost_impact:.1%}")


单边交易成本：12 bps
年化换手率：6.5x
总成本影响：8.3%


## 4. 绩效指标计算与对比

**目的**：计算三种回测方法的关键绩效指标，量化前视偏差和成本的影响。

**金融含义**：年化收益、波动率、夏普比率、最大回撤是衡量策略优劣的核心指标。通过对比可以看出前视偏差会虚增收益，而交易成本会显著降低实际可获得的收益。


In [31]:
def calc_metrics(returns, name):
    """计算策略绩效指标"""
    equity = (1 + returns).cumprod()
    
    # 年化收益
    cagr = (equity.iloc[-1] ** (252 / len(returns))) - 1
    
    # 年化波动
    vol = returns.std(ddof=0) * np.sqrt(252)
    
    # 夏普比率 - 修复 Series 布尔值判断问题
    std_val = returns.std(ddof=0)
    mean_val = returns.mean()
    sharpe = np.sqrt(252) * mean_val / std_val if std_val > 0 else 0
    
    # 最大回撤
    max_dd = ((equity / equity.cummax()) - 1).min()
    
    return {
        '策略': name,
        '年化收益': f"{cagr:.1%}",
        '年化波动': f"{vol:.1%}",
        '夏普比率': f"{sharpe:.2f}",
        '最大回撤': f"{max_dd:.1%}"
    }

print(ret_naive)
# 修复收益率序列问题 - 确保都是 Series 格式
ret_naive_series = ret_naive if isinstance(ret_naive, pd.Series) else ret_naive.iloc[:, 0]
ret_correct_series = ret_correct if isinstance(ret_correct, pd.Series) else ret_correct.iloc[:, 0]

# 计算三种方法的指标
metrics = [
    calc_metrics(ret_naive_series, '前视偏差'),
    calc_metrics(ret_correct_series, '正确回测'), 
    calc_metrics(ret_correct_cost, '含交易成本')
]

metrics_df = pd.DataFrame(metrics)
print("\n=== 回测结果对比 ===")
print(metrics_df.to_string(index=False))


Ticker          AAPL
Date                
2015-01-02  0.000000
2015-01-05 -0.000000
2015-01-06  0.000000
2015-01-07  0.000000
2015-01-08  0.000000
...              ...
2025-09-02 -0.010425
2025-09-03  0.038090
2025-09-04  0.005493
2025-09-05 -0.000375
2025-09-08 -0.007551

[2686 rows x 1 columns]


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

## 5. 净值曲线可视化

**目的**：绘制三条净值曲线的对比图，直观展示前视偏差和交易成本的影响。

**金融含义**：净值曲线是评估策略表现最直观的方式。图中可以清楚看到前视偏差导致的虚假优势，以及交易成本对收益的侵蚀程度。


In [26]:
# 绘制净值曲线对比
plt.figure(figsize=(12, 6))
plt.plot(equity_naive.index, equity_naive.values, label='前视偏差回测', linewidth=2, alpha=0.8)
plt.plot(equity_correct.index, equity_correct.values, label='正确回测', linewidth=2, alpha=0.8)
plt.plot(equity_correct_cost.index, equity_correct_cost.values, label='含交易成本', linewidth=2, alpha=0.8)

plt.title('AAPL动量策略：前视偏差与交易成本影响', fontsize=14, fontweight='bold')
plt.xlabel('日期', fontsize=12)
plt.ylabel('累计净值', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# 计算收益差异
final_naive = equity_naive.iloc[-1]
final_correct = equity_correct.iloc[-1]
final_cost = equity_correct_cost.iloc[-1]

print(f"\n=== 最终净值对比 ===")
print(f"前视偏差回测：{final_naive:.3f} (+{final_naive-1:.1%})")
print(f"正确回测：{final_correct:.3f} (+{final_correct-1:.1%})")
print(f"含交易成本：{final_cost:.3f} (+{final_cost-1:.1%})")
print(f"\n前视偏差高估收益：{(final_naive/final_correct-1):.1%}")
print(f"交易成本侵蚀收益：{(final_correct/final_cost-1):.1%}")


C:\Users\zyf17\AppData\Local\Temp\ipykernel_14052\41638809.py:12: UserWarning: Glyph 26085 (\N{CJK UNIFIED IDEOGRAPH-65E5}) missing from current font.
  plt.tight_layout()
C:\Users\zyf17\AppData\Local\Temp\ipykernel_14052\41638809.py:12: UserWarning: Glyph 26399 (\N{CJK UNIFIED IDEOGRAPH-671F}) missing from current font.
  plt.tight_layout()
C:\Users\zyf17\AppData\Local\Temp\ipykernel_14052\41638809.py:12: UserWarning: Glyph 32047 (\N{CJK UNIFIED IDEOGRAPH-7D2F}) missing from current font.
  plt.tight_layout()
C:\Users\zyf17\AppData\Local\Temp\ipykernel_14052\41638809.py:12: UserWarning: Glyph 35745 (\N{CJK UNIFIED IDEOGRAPH-8BA1}) missing from current font.
  plt.tight_layout()
C:\Users\zyf17\AppData\Local\Temp\ipykernel_14052\41638809.py:12: UserWarning: Glyph 20928 (\N{CJK UNIFIED IDEOGRAPH-51C0}) missing from current font.
  plt.tight_layout()
C:\Users\zyf17\AppData\Local\Temp\ipykernel_14052\41638809.py:12: UserWarning: Glyph 20540 (\N{CJK UNIFIED IDEOGRAPH-503C}) missing from cur


=== 最终净值对比 ===


TypeError: unsupported format string passed to Series.__format__

## 6. 限价单填单逻辑演示

**目的**：演示简单的限价单逻辑，对比市价单和限价单的执行效果。

**金融含义**：限价单可以获得更好的成交价格，但存在无法成交的风险。在量化交易中需要权衡价格改善和执行确定性。逢低买入策略通过限价单可以降低成本，但可能错过部分交易机会。


In [ ]:
# 限价单逻辑：逢低买入
data_full = yf.download('AAPL', start='2015-01-01', end=datetime.now().strftime('%Y-%m-%d'), auto_adjust=True)
high = data_full['High']
low = data_full['Low']
close = data_full['Close']

# 找到需要开仓的点位（0->1）
entry_signals = (signal.diff() == 1)
limit_discount = 0.002  # 限价单折扣0.2%

# 计算限价单执行情况
pos_limit = pos.copy()
fill_rate = 0
total_signals = 0

for i in range(1, len(signal)):
    if entry_signals.iloc[i]:  # 需要开仓
        total_signals += 1
        limit_price = close.iloc[i-1] * (1 - limit_discount)
        
        # 检查次日是否能成交
        if i < len(low) and low.iloc[i] <= limit_price <= high.iloc[i]:
            fill_rate += 1
        else:
            pos_limit.iloc[i] = 0  # 未成交，保持空仓

fill_rate = fill_rate / total_signals if total_signals > 0 else 0

# 计算限价单策略收益
ret_limit = pos_limit * ret
equity_limit = (1 + ret_limit).cumprod()

print(f"\n=== 限价单 vs 市价单对比 ===")
print(f"总开仓信号数：{total_signals}")
print(f"限价单填单率：{fill_rate:.1%}")
print(f"市价单最终收益：{(final_correct-1):.1%}")
print(f"限价单最终收益：{(equity_limit.iloc[-1]-1):.1%}")


## 7. 幸存者偏差演示

**目的**：用合成数据演示幸存者偏差对回测结果的影响。

**金融含义**：幸存者偏差是指只考虑"存活"到研究期末的标的，忽略了中途退市或表现极差的标的。这会系统性高估策略收益。实际研究中应使用当期成分股快照，避免这种偏差。


In [ ]:
# 幸存者偏差实验
np.random.seed(42)
n_assets = 100
n_days = 252 * 5  # 5年数据

# 生成100只"股票"的收益率
daily_returns = np.random.normal(0.0005, 0.02, (n_days, n_assets))  # 日均0.05%收益，2%波动

# 计算累计收益
cumret = (1 + daily_returns).cumprod(axis=0)
final_returns = cumret[-1, :] - 1

# 原始平均收益（所有股票）
avg_return_all = final_returns.mean()

# 剔除最差20%后的平均收益（幸存者偏差）
survivor_mask = final_returns >= np.percentile(final_returns, 20)
avg_return_survivors = final_returns[survivor_mask].mean()

bias_impact = avg_return_survivors - avg_return_all

print(f"\n=== 幸存者偏差实验 ===")
print(f"全部100只股票平均收益：{avg_return_all:.1%}")
print(f"剔除最差20%后平均收益：{avg_return_survivors:.1%}")
print(f"幸存者偏差影响：+{bias_impact:.1%}")
print(f"\n注意：真实研究中应使用当期成分股快照，避免此类偏差")


## 8. 成本影响可视化

**目的**：用条形图展示各项成本对策略收益的具体影响。

**金融含义**：将成本细分有助于识别优化点。佣金相对固定，而滑点和价差与市场流动性相关，可通过改善执行算法来降低。


In [ ]:
# 成本构成分析
cost_breakdown = {
    '佣金成本': turnover.sum() * (commission_bps / 1e4),
    '滑点成本': turnover.sum() * (slippage_bps / 1e4),
    '价差成本': turnover.sum() * (half_spread_bps / 1e4)
}

plt.figure(figsize=(10, 6))
costs = list(cost_breakdown.values())
labels = list(cost_breakdown.keys())
colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']

bars = plt.bar(labels, [c*100 for c in costs], color=colors, alpha=0.7)
plt.title('交易成本构成分析', fontsize=14, fontweight='bold')
plt.ylabel('成本影响 (%)', fontsize=12)
plt.grid(True, alpha=0.3, axis='y')

# 添加数值标签
for bar, cost in zip(bars, costs):
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height + 0.01,
             f'{cost:.2%}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n总交易成本：{sum(costs):.2%}")
print(f"成本占策略收益比例：{sum(costs)/(final_correct-1):.1%}")


## 要点总结

### 前视偏差（Look-ahead Bias）
- **错误做法**：用当日信号交易当日收益
- **正确做法**：用前一日信号决定今日仓位（signal.shift(1)）
- **影响**：前视偏差会系统性高估策略收益

### 交易成本构成
- **佣金**：券商收取的固定费用
- **滑点**：市场冲击导致的价格偏移
- **买卖价差**：bid-ask spread的成本
- **关键**：成本与换手率成正比，高频策略更敏感

### 换手率与成本
- 换手率 = 仓位变化的绝对值
- 年化换手率 = 日换手率之和 × 252
- 成本侵蚀 = 换手率 × 单次交易成本

### 幸存者偏差注意事项
- **问题**：只考虑"存活"标的，忽略退市/表现差的
- **解决**：使用当期成分股快照进行回测
- **重要性**：避免系统性高估历史收益表现

### 实盘考虑因素
- 限价单可改善成交价格但存在执行风险
- 流动性约束会影响大资金的实际执行
- 市场微观结构对高频策略影响更大
